# This Notebook defines and review the Vertex CNN model 

In [1]:
from tensorflow.keras.regularizers import l1
import tensorflow.compat.v2 as tf
from tensorflow import keras
import qkeras

### Standard ConV Block + Dense Block CNN  (BaseLine Model)

In [ ]:
def std_vertex_cnn(
             
    input_shape = (56,200,1),             
    output_size = 2,
    filters_per_conv_layer = [], # int in list
    kernels_per_conv_layer = [], # tuple in list
    strides_per_conv_layer = [], # tuple in list
    poolsize_per_conv_layer = [],# tuple in list
    neurons_per_dense_layer = [] # int in list
):

    # Input Layer
    x = x_in = keras.layers.Input(input_shape)
    
    # Conv Block
    for layer,filters in enumerate(filters_per_conv_layer):
        
        filter_num = filters
        filter_kernel_size = kernels_per_conv_layer[layer]
        filter_strides = strides_per_conv_layer[layer]
        poolsize = poolsize_per_conv_layer[layer]

        x = keras.layers.Conv2D(filter_num,
                                filter_kernel_size,
                                strides = filter_strides,
                                kernel_initializer = "lecun_uniform",
                                kernel_regularizer = l1(0.0001),
                                use_bias = False,
                                name = f"conv_{layer}"
                               )(x)
        x = keras.layers.BatchNormalization(name = f"conv_bn_{layer}")(x)
        x = keras.layers.Activation("relu",name = f"conv_act_{layer}")(x)
        x = keras.layers.MaxPooling2D(pool_size=poolsize,name = f"conv_pool_{layer}")(x)
        
    
    # Flatten layer    
    x = keras.layers.Flatten()(x)
    
    # Dense Block
    for layer,neurons in enumerate(neurons_per_dense_layer):

        x = keras.layers.Dense(neurons,
                               kernel_initializer = "lecun_uniform",
                               kernel_regularizer = l1(0.0001),
                               name = f"dense_{layer}",
                               use_bias = False)(x)
        x = keras.layers.BatchNormalization(name = f"dense_bn_{layer}")(x)
        x = keras.layers.Activation("relu",name = f"dense_act_{layer}")(x)
    
    # Output Dense layer
    x_out = x = keras.layers.Dense(output_size,name = "dense_output")(x)
    model = keras.models.Model(inputs = x_in,
                               outputs = x,
                               )
    return model 

### Quantized ConV Block + Dense Block CNN 

In [2]:
def create_vertex_Qcnn(
             
    input_shape = (56,200,1),             
    output_size = 2,
    
    # Conv 2d
    filters_per_conv_layer = [],   # int in list
    kernels_per_conv_layer = [],   # tuple in list
    strides_per_conv_layer = [],   # tuple in list
    poolsize_per_conv_layer = [],  # tuple in list   

    # Conv 2d Quantization
    conv_kernel_quantizer = [],    # string in list
    conv_bias_quantizer =   [],    # string in list
    conv_activatoin_quantizer = [],
    
    
    # Dense 
    neurons_per_dense_layer = [],  # int in list,
    
    # Dense Quantization
    dense_weight_quantizer = [],   # string in list
    dense_bias_quantizer  = [],     # string in list
    dense_activation_quantizer = [], #string in list
    
    dense_output_weight_quantizer = "",
    dense_output_bias_quantizer = ""
):

    # Input Layer
    
    x = x_in = keras.layers.Input(input_shape,name = "input")
    
    # Conv Block
    for layer,filters in enumerate(filters_per_conv_layer):
        
        filter_num = filters
        filter_kernel_size = kernels_per_conv_layer[layer]
        filter_strides = strides_per_conv_layer[layer]
        poolsize = poolsize_per_conv_layer[layer]
               
        x = qkeras.QConv2D(     filter_num,
                                filter_kernel_size,
                                strides = filter_strides,
                                kernel_initializer = "lecun_uniform",
                                kernel_regularizer = l1(0.0001),
                                use_bias = False,
                                name = f"conv_{layer}",
                           
                                kernel_quantizer = conv_kernel_quantizer[layer], 
                                bias_quantizer = conv_bias_quantizer[layer] 
                               )(x)
        
        x = qkeras.QActivation(conv_activatoin_quantizer[layer])(x)
        
        x = keras.layers.MaxPooling2D(pool_size = poolsize,
                                          name = f"conv_pool_{layer}")(x)
        
    
    # Flatten layer   
    if len(filters_per_conv_layer) != 0:
        x = keras.layers.Flatten()(x)
    
    # Dense Block
    for layer,neurons in enumerate(neurons_per_dense_layer):

        x = qkeras.QDense(neurons,
                               kernel_initializer = "random_normal",
                               #kernel_regularizer = l1(0.1),
                               name = f"dense_{layer}",
                               use_bias = True,
                          
                               kernel_quantizer = dense_weight_quantizer[layer], 
                               bias_quantizer   = dense_bias_quantizer[layer])(x)
        
        #x = keras.layers.BatchNormalization(name = f"dense_bn_{layer}")(x)
        
        x = qkeras.QActivation(dense_activation_quantizer[layer],name = f"dense_act_{layer}")(x)
    
    # Output Dense layer
    #x_out = x = keras.layers.Dense(output_size,name = "dense_output",use_bias = False)(x)
    x_out = x = qkeras.QDense( output_size,
                               kernel_initializer = "random_normal",
                               #kernel_regularizer = l1(0.0001),
                               name = "dense_output",
                               use_bias = False,
                          
                               kernel_quantizer = dense_output_weight_quantizer, 
                               bias_quantizer   = dense_output_bias_quantizer)(x)
    model = keras.models.Model(inputs = x_in,
                               outputs = x_out,
                               name = "Qmodel"
                               )
    return model 
